In [7]:
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from ucimlrepo import fetch_ucirepo 
  

In [8]:

# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features[:800] 
y = adult.data.targets[:800]
  
# # metadata 
# print(adult.metadata) 
  
# # variable information 
# print(adult.variables) 


In [11]:
X.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,800.000000,8.000000e+02,800.000000,800.000000,800.000000,800.000000
mean,37.732500,1.925954e+05,10.098750,631.106250,97.763750,39.772500
std,13.293677,1.088143e+05,2.569485,2768.048418,420.372087,11.800734
min,17.000000,2.117400e+04,1.000000,0.000000,0.000000,1.000000
25%,27.000000,1.150412e+05,9.000000,0.000000,0.000000,40.000000
50%,36.000000,1.835455e+05,10.000000,0.000000,0.000000,40.000000
75%,46.000000,2.471522e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.033222e+06,16.000000,34095.000000,2415.000000,98.000000


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(1, 11), accuracies)
plt.xlabel('k')
plt.ylabel('Accuracy')
plt.title('k-NN Varying number of neighbors')
plt.show()